Testin the tapas-large-finetuned-wtq model on the data

In [10]:
import pandas as pd
from transformers import pipeline

# Load the CSV file
file_path = "../data/PARAMETROS_TJ2_ORDENADOS.csv"  # Replace with your file path
data = pd.read_csv(file_path, delimiter=";", encoding="latin1")

# Handle missing values and convert all columns to strings
data.fillna("N/A", inplace=True)
data = data.astype(str)

# Reduce table size to relevant columns if needed
columns_to_keep = ["N_DESCARGA", "fecha", "hora", "comentarioDesc"]
data = data[columns_to_keep]

# Load the TAPAS pipeline with the base model
pipe = pipeline("table-question-answering", model="google/tapas-base-finetuned-wtq", device=-1)

# Define questions
questions = [
    "CUAL ES LA FECHA PARA EL 4?",
    "What is the hora for N_DESCARGA 5?",
    "What is the comentarioDesc for N_DESCARGA 12?"
]

# Process each question
print("\nResults:")
for question in questions:
    # Dynamically filter rows relevant to the question (if possible)
    try:
        # Filter rows dynamically based on the query
        query_value = 4  # Example value extracted from the question
        filtered_table = data[data["N_DESCARGA"] == str(query_value)]  # Reduce the table to relevant rows
    except ValueError:
        filtered_table = data  # Default to the full table if no filtering is possible

    # Run the pipeline with the filtered table
    answer = pipe(table=filtered_table, query=question)
    print(f"Question: {question}")
    print(f"Answer: {answer['answer']}\n")

/var/folders/b6/pb_stfjn0pb7pybgwlqjk5700000gn/T/ipykernel_47698/2548006602.py:6: DtypeWarning: Columns (52,54,55,96,105,108,110,122,125,129,132,133,137,138,143,144,145,150,151,154) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, delimiter=";", encoding="latin1")
/var/folders/b6/pb_stfjn0pb7pybgwlqjk5700000gn/T/ipykernel_47698/2548006602.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'N/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.fillna("N/A", inplace=True)
Device set to use cpu



Results:
Question: CUAL ES LA FECHA PARA EL 4?
Answer: 19971126.0

Question: What is the hora for N_DESCARGA 5?
Answer: 00:00

Question: What is the comentarioDesc for N_DESCARGA 12?
Answer: N/A



/Users/sofiamorenolasa/Desktop/TFG/.venv/lib/python3.10/site-packages/transformers/models/tapas/tokenization_tapas.py:2699: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/Users/sofiamorenolasa/Desktop/TFG/.venv/lib/python3.10/site-packages/transformers/models/tapas/tokenization_tapas.py:1493: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


Now let´s see if it works if we ask for a bigger number. 

Now we will try using a binary search algorithm to efficiently locate the N_DESCARGA value in the table, and try to reduce the search time. 

I am going to try a different approach: use a model to convert the question into SQL and query the dataset directly.

Loggin using huggingface-cli login, and put your access token.

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import pickle

# Load the model and tokenizer from Hugging Face
model_name = "meta-llama/Llama-2-70b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Create a text generation pipeline
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Save the pipeline as a pickle file
with open("llm_pipeline.pkl", "wb") as f:
    pickle.dump(llm_pipeline, f)

print("Pipeline saved as llm_pipeline.pkl")

KeyboardInterrupt: 

In [18]:
import pandas as pd
from langchain_community.llms import Replicate
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Set up Replicate for LLaMA-2
os.environ["REPLICATE_API_TOKEN"] = os.getenv("REPLICATE_API_TOKEN")
llama2_13b_chat = "meta/llama-2-7b-chat"

llm = Replicate(
    model=llama2_13b_chat,
    model_kwargs={"temperature": 0.7, "max_new_tokens": 100}
)

# Load the CSV file
file_path = "../data/PARAMETROS_TJ2_ORDENADOS.csv"  # Replace with your file path
data = pd.read_csv(file_path, delimiter=";", encoding="latin1", low_memory=False)

# Ensure missing values are replaced properly
for column in data.columns:
    if data[column].dtype == "float64":
        data[column] = data[column].fillna(-1)
        if column == "N_DESCARGA":
            data[column] = data[column].astype(int)
    else:
        data[column] = data[column].fillna("N/A")

# Convert the entire DataFrame to strings
data = data.astype(str)

# Define the column names and script as context
script_context = (
    "The table is named 'data' and contains the following important columns:\n"
    "N_DESCARGA, fecha, hora, comentarioDesc, comentarioExp, configuracion, "
    "potencia_radiada, energia_diamagnetica.\n"
    "You must use these column names exactly as they are when writing SQL queries.\n"
    "Always use 'N_DESCARGA' as the column for filtering by number.\n"
    "Respond with only the SQL query, without any explanation or additional text.\n"
)

# Define questions
questions = [
    "What is the fecha for N_DESCARGA 4?",
    "What is the hora for N_DESCARGA 26458?",
    "What is the comentarioDesc for N_DESCARGA 12?"
]

# Helper function to execute SQL queries on the DataFrame
def execute_sql_query(data, sql_query):
    try:
        # Use pandasql to execute SQL queries
        import pandasql as ps
        result = ps.sqldf(sql_query, locals())
        return result
    except Exception as e:
        return f"SQL Execution Error: {e}"

# Helper function to validate SQL query
def is_valid_sql(sql_query):
    return sql_query.strip().upper().startswith("SELECT")

# Process each question
print("\nResults:")
for question in questions:
    try:
        # Provide the script and the question to the LLM
        llm_input = f"{script_context}\nConvert the following question into an SQL query: {question}"
        
        # Get the generated SQL query from LLaMA-2
        response = llm.invoke(input=llm_input).strip()  # Extract and clean the response

        # Debugging: Print the generated SQL query
        print(f"Generated SQL Query: {response}")

        # Validate the SQL query
        if not is_valid_sql(response):
            print(f"Invalid SQL query generated for question: {question}")
            continue

        # Execute the SQL query on the DataFrame
        result = execute_sql_query(data, response)

        # Output the result
        print(f"Question: {question}")
        print(f"Answer: {result}\n")

    except Exception as e:
        print(f"Error during processing for question '{question}': {e}\n")


Results:
Generated SQL Query: SELECT fecha FROM data WHERE N_DESCARGA = 4;
Question: What is the fecha for N_DESCARGA 4?
Answer:         fecha
0  19971126.0

Generated SQL Query: SELECT hora FROM data WHERE N_DESCARGA = 26458;
Question: What is the hora for N_DESCARGA 26458?
Answer:     hora
0  14:14

Generated SQL Query: SELECT comentarioDesc FROM data WHERE N_DESCARGA = 12;
Question: What is the comentarioDesc for N_DESCARGA 12?
Answer:   comentarioDesc
0            N/A



In [19]:
import pandas as pd
from langchain_community.llms import Replicate
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Set up Replicate for LLaMA-2
os.environ["REPLICATE_API_TOKEN"] = os.getenv("REPLICATE_API_TOKEN")
llama2_13b_chat = "meta/llama-2-7b-chat"

llm = Replicate(
    model=llama2_13b_chat,
    model_kwargs={"temperature": 0.7, "max_new_tokens": 100}
)

# Load the CSV file
file_path = "../data/PARAMETROS_TJ2_ORDENADOS.csv"  # Replace with your file path
data = pd.read_csv(file_path, delimiter=";", encoding="latin1", low_memory=False)

# Ensure missing values are replaced properly
for column in data.columns:
    if data[column].dtype == "float64":
        data[column] = data[column].fillna(-1)
        if column == "N_DESCARGA":
            data[column] = data[column].astype(int)
    else:
        data[column] = data[column].fillna("N/A")

# Convert the entire DataFrame to strings
data = data.astype(str)

# Define the column names and script as context
script_context = (
    "The table is named 'data' and contains the following important columns:\n"
    "N_DESCARGA, fecha, hora, comentarioDesc, comentarioExp, configuracion, "
    "potencia_radiada, energia_diamagnetica.\n"
    "You must use these column names exactly as they are when writing SQL queries.\n"
    "Always use 'N_DESCARGA' as the column for filtering by number.\n"
    "Respond with only the SQL query, without any explanation or additional text.\n"
)

# Define questions
questions = [
    "Cual es la fecha para el numero de descarga 4?",
    "cual es la hora para el numero de descarga 26458?",
    "cual es el comentario para el numero de descarga 12?"
]

# Helper function to execute SQL queries on the DataFrame
def execute_sql_query(data, sql_query):
    try:
        # Use pandasql to execute SQL queries
        import pandasql as ps
        result = ps.sqldf(sql_query, locals())
        return result
    except Exception as e:
        return f"SQL Execution Error: {e}"

# Helper function to validate SQL query
def is_valid_sql(sql_query):
    return sql_query.strip().upper().startswith("SELECT")

# Process each question
print("\nResults:")
for question in questions:
    try:
        # Provide the script and the question to the LLM
        llm_input = f"{script_context}\nConvert the following question into an SQL query: {question}"
        
        # Get the generated SQL query from LLaMA-2
        response = llm.invoke(input=llm_input).strip()  # Extract and clean the response

        # Debugging: Print the generated SQL query
        print(f"Generated SQL Query: {response}")

        # Validate the SQL query
        if not is_valid_sql(response):
            print(f"Invalid SQL query generated for question: {question}")
            continue

        # Execute the SQL query on the DataFrame
        result = execute_sql_query(data, response)

        # Output the result
        print(f"Question: {question}")
        print(f"Answer: {result}\n")

    except Exception as e:
        print(f"Error during processing for question '{question}': {e}\n")


Results:
Generated SQL Query: SELECT fecha FROM data WHERE N_DESCARGA = 4;
Question: Cual es la fecha para el numero de descarga 4?
Answer:         fecha
0  19971126.0

Generated SQL Query: Sure, here is the SQL query to answer the question:

SELECT hora
FROM data
WHERE N_DESCARGA = 26458;
Invalid SQL query generated for question: cual es la hora para el numero de descarga 26458?
Generated SQL Query: Sure, I'd be happy to help! Here is the SQL query to answer the question:

SELECT comentarioDesc FROM data WHERE N_DESCARGA = 12;
Invalid SQL query generated for question: cual es el comentario para el numero de descarga 12?


Testing with different quesitons

In [ ]:
import pandas as pd
from langchain_community.llms import Replicate
from dotenv import load_dotenv
from googletrans import Translator
import asyncio
import os
import nest_asyncio
nest_asyncio.apply()

# Load environment variables
load_dotenv()

# Set up Replicate for LLaMA-2
os.environ["REPLICATE_API_TOKEN"] = os.getenv("REPLICATE_API_TOKEN")
llama2_13b_chat = "meta/llama-2-7b-chat"

llm = Replicate(
    model=llama2_13b_chat,
    model_kwargs={"temperature": 0.7, "max_new_tokens": 100}
)

# Load the CSV file
file_path = "../data/PARAMETROS_TJ2_ORDENADOS.csv"  # Replace with your file path
data = pd.read_csv(file_path, delimiter=";", encoding="latin1", low_memory=False)

# Ensure missing values are replaced properly
for column in data.columns:
    if data[column].dtype == "float64":
        data[column] = data[column].fillna(-1)
        if column == "N_DESCARGA":
            data[column] = data[column].astype(int)
    else:
        data[column] = data[column].fillna("N/A")

# Convert the entire DataFrame to strings
data = data.astype(str)

# Define the column names and script as context
script_context = (
    "The table is named 'data' and contains the following important columns:\n"
    "N_DESCARGA, fecha, hora, comentarioDesc, comentarioExp, configuracion, "
    "potencia_radiada, energia_diamagnetica.\n"
    "You must use these column names exactly as they are when writing SQL queries.\n"
    "Always use 'N_DESCARGA' as the column for filtering by number.\n"
    "Questions may be in Spanish or English, but the output must always be a valid SQL query.\n"
    "Do not include any explanations, greetings, or additional text in your response. Only output the SQL query."
)

# Spanish questions
questions = [
    "Cual es la fecha para el numero de descarga 4?",
    "cual es la hora para el numero de descarga 26458?",
    "cual es el comentario para el numero de descarga 12?"
]

# Helper function to execute SQL queries on the DataFrame
def execute_sql_query(data, sql_query):
    try:
        # Use pandasql to execute SQL queries
        import pandasql as ps
        result = ps.sqldf(sql_query, locals())
        return result
    except Exception as e:
        return f"SQL Execution Error: {e}"

# Helper function to validate SQL query
def is_valid_sql(sql_query):
    return sql_query.strip().upper().startswith("SELECT")

# Translate questions and process them
async def process_questions():
    translator = Translator()

    print("\nResults:")
    for question in questions:
        try:
            # Translate the question to English
            translated_question = await translator.translate(question, src='es', dest='en')

            # Provide the script and the translated question to the LLM
            llm_input = f"{script_context}\nConvert the following question into an SQL query: {translated_question.text}"

            # Get the generated SQL query from LLaMA-2
            response = llm.invoke(input=llm_input).strip()  # Extract and clean the response

            # Debugging: Print the generated SQL query
            print(f"Generated SQL Query: {response}")

            # Validate the SQL query
            if not is_valid_sql(response):
                print(f"Invalid SQL query generated for question: {question}")
                continue

            # Execute the SQL query on the DataFrame
            result = execute_sql_query(data, response)

            # Output the result
            print(f"Question (Original): {question}")
            print(f"Question (Translated): {translated_question.text}")
            print(f"Answer: {result}\n")

        except Exception as e:
            print(f"Error during processing for question '{question}': {e}\n")

# Run the processing function in an existing event loop
try:
    loop = asyncio.get_running_loop()
    task = loop.create_task(process_questions())
    loop.run_until_complete(task)
except RuntimeError:
    asyncio.run(process_questions())


Results:
Generated SQL Query: SELECT fecha FROM data WHERE N_DESCARGA = 4;
Question (Original): Cual es la fecha para el numero de descarga 4?
Question (Translated): What is the date for download number 4?
Answer:         fecha
0  19971126.0

Generated SQL Query: Sure, I'd be happy to help! Here is the SQL query to answer the question:

SELECT hora FROM data WHERE N_DESCARGA = 26458;
Invalid SQL query generated for question: cual es la hora para el numero de descarga 26458?
Generated SQL Query: SELECT comentarioDesc FROM data WHERE N_DESCARGA = 12;
Question (Original): cual es el comentario para el numero de descarga 12?que numero de descarga tiene este comentario: RAYOS - X
Question (Translated): What is the comment for download number 12? What download number does this comment have: RAYOS - X
Answer:   comentarioDesc
0            N/A



In [36]:
import pandas as pd
from langchain_community.llms import Replicate
from dotenv import load_dotenv
from googletrans import Translator
import asyncio
import os
import nest_asyncio
nest_asyncio.apply()

# Load environment variables
load_dotenv()

# Set up Replicate for LLaMA-2
os.environ["REPLICATE_API_TOKEN"] = os.getenv("REPLICATE_API_TOKEN")
llama2_13b_chat = "meta/llama-2-7b-chat"

llm = Replicate(
    model=llama2_13b_chat,
    model_kwargs={"temperature": 0.7, "max_new_tokens": 100}
)

# Load the CSV file
file_path = "../data/PARAMETROS_TJ2_ORDENADOS.csv"  # Replace with your file path
data = pd.read_csv(file_path, delimiter=";", encoding="latin1", low_memory=False)

# Ensure missing values are replaced properly
for column in data.columns:
    if data[column].dtype == "float64":
        data[column] = data[column].fillna(-1)
        if column == "N_DESCARGA":
            data[column] = data[column].astype(int)
    else:
        data[column] = data[column].fillna("N/A")

# Convert the entire DataFrame to strings
data = data.astype(str)

# Define the column names and script as context
script_context = (
    "The table is named 'data' and contains the following important columns:\n"
    "N_DESCARGA, fecha, hora, comentarioDesc, comentarioExp, configuracion, "
    "potencia_radiada, energia_diamagnetica.\n"
    "You must use these column names exactly as they are when writing SQL queries.\n"
    "Always use 'N_DESCARGA' as the column for filtering by number.\n"
    "Questions may be in Spanish or English, but the output must always be a valid SQL query.\n"
    "Do not include any explanations, greetings, or additional text in your response. Only output the SQL query."
)

# Spanish questions
questions = [
    "Cual es la fecha para el numero de descarga 4?",
    "cual es la hora para el numero de descarga 26458?",
    "cual es el comentario para el numero de descarga 8621?"
]

# Helper function to execute SQL queries on the DataFrame
def execute_sql_query(data, sql_query):
    try:
        # Use pandasql to execute SQL queries
        import pandasql as ps
        result = ps.sqldf(sql_query, locals())
        return result
    except Exception as e:
        return f"SQL Execution Error: {e}"

# Translate questions and process them
async def process_questions():
    translator = Translator()

    print("\nResults:")
    for question in questions:
        try:
            # Translate the question to English
            translated_question = await translator.translate(question, src='es', dest='en')

            # Provide the script and the translated question to the LLM
            llm_input = f"{script_context}\nConvert the following question into an SQL query: {translated_question.text}"

            # Get the generated SQL query from LLaMA-2
            response = llm.invoke(input=llm_input).strip()  # Extract and clean the response

            # Validate the SQL query
            if not response.strip().upper().startswith("SELECT"):
                print(f"Invalid SQL query generated for question: {question}")
                continue

            # Execute the SQL query on the DataFrame
            result = execute_sql_query(data, response)

            # Output only the question and the result
            print(f"Question (Original): {question}")
            print(f"Question (Translated): {translated_question.text}")
            print(f"Answer: {result}\n")

        except Exception as e:
            print(f"Error during processing for question '{question}': {e}\n")

# Run the processing function in an existing event loop
try:
    loop = asyncio.get_running_loop()
    task = loop.create_task(process_questions())
    loop.run_until_complete(task)
except RuntimeError:
    asyncio.run(process_questions())


Results:
Question (Original): Cual es la fecha para el numero de descarga 4?
Question (Translated): What is the date for download number 4?
Answer:         fecha
0  19971126.0

Question (Original): cual es la hora para el numero de descarga 26458?
Question (Translated): What is the time for download number 26458?
Answer:     hora
0  14:14

Question (Original): cual es el comentario para el numero de descarga 8621?
Question (Translated): What is the comment for download number 8621?
Answer:                                       comentarioDesc
0  LA DENSIDAD LLEGA HASTA 0.75. RAYOS X. PARA EL...



Testing with more questions